In [139]:
import numpy as np
import gymnasium as gym
import torch.nn as nn
import torch

import sys
import os

sys.path.append(os.path.abspath(".."))

import rlib

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [144]:
env = gym.make("CartPole-v1")

In [145]:
dqn_agent = rlib.DQN(env=env, batch_size=64)

In [146]:
dqn_agent.train(episode_n=100)

  8%|▊         | 16/200 [00:00<00:01, 146.81it/s]

Episode: 0, Total Episode Reward: 9.0
Episode: 1, Total Episode Reward: 9.0
Episode: 2, Total Episode Reward: 9.0
Episode: 3, Total Episode Reward: 9.0
Episode: 4, Total Episode Reward: 10.0
Episode: 5, Total Episode Reward: 10.0
Episode: 6, Total Episode Reward: 9.0
Episode: 7, Total Episode Reward: 9.0
Episode: 8, Total Episode Reward: 9.0
Episode: 9, Total Episode Reward: 8.0
Episode: 10, Total Episode Reward: 9.0
Episode: 11, Total Episode Reward: 10.0
Episode: 12, Total Episode Reward: 9.0
Episode: 13, Total Episode Reward: 9.0
Episode: 14, Total Episode Reward: 10.0
Episode: 15, Total Episode Reward: 9.0
Episode: 16, Total Episode Reward: 10.0
Episode: 17, Total Episode Reward: 9.0
Episode: 18, Total Episode Reward: 8.0
Episode: 19, Total Episode Reward: 10.0
Episode: 20, Total Episode Reward: 10.0
Episode: 21, Total Episode Reward: 10.0
Episode: 22, Total Episode Reward: 10.0
Episode: 23, Total Episode Reward: 9.0


 16%|█▌        | 31/200 [00:00<00:01, 109.22it/s]

Episode: 24, Total Episode Reward: 10.0
Episode: 25, Total Episode Reward: 11.0
Episode: 26, Total Episode Reward: 10.0
Episode: 27, Total Episode Reward: 10.0
Episode: 28, Total Episode Reward: 9.0
Episode: 29, Total Episode Reward: 8.0
Episode: 30, Total Episode Reward: 9.0
Episode: 31, Total Episode Reward: 8.0
Episode: 32, Total Episode Reward: 10.0
Episode: 33, Total Episode Reward: 11.0
Episode: 34, Total Episode Reward: 10.0
Episode: 35, Total Episode Reward: 10.0
Episode: 36, Total Episode Reward: 12.0
Episode: 37, Total Episode Reward: 9.0
Episode: 38, Total Episode Reward: 11.0
Episode: 39, Total Episode Reward: 10.0
Episode: 40, Total Episode Reward: 9.0
Episode: 41, Total Episode Reward: 10.0


 22%|██▏       | 43/200 [00:00<00:01, 98.62it/s] 

...

 99%|█████████▉| 198/200 [01:06<00:00,  5.43it/s]

Episode: 196, Total Episode Reward: 70.0
Episode: 197, Total Episode Reward: 68.0


100%|██████████| 200/200 [01:07<00:00,  2.98it/s]

Episode: 198, Total Episode Reward: 73.0
Episode: 199, Total Episode Reward: 67.0


[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0